## Import Neccesary Packages

In [136]:
%load_ext autoreload
%autoreload 2

import syft as sy
from syft.serde import protobuf
from syft_proto.execution.v1.plan_pb2 import Plan as PlanPB
from syft_proto.execution.v1.state_pb2 import State as StatePB
from syft.grid.clients.model_centric_fl_client import ModelCentricFLClient
from syft.execution.state import State
from syft.execution.placeholder import PlaceHolder
from syft.execution.translation import TranslationTarget

import torch as th
from torch import nn

import os
from websocket import create_connection
import websockets
import json
import requests

sy.make_hook(globals())
hook.local_worker.framework = None # force protobuf serialization for tensors
th.random.manual_seed(1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Setting up Sandbox...
Done!


In [137]:
def set_model_params(module, params_list, start_param_idx=0):
    """ Set params list into model recursively
    """
    param_idx = start_param_idx

    for name, param in module._parameters.items():
        module._parameters[name] = params_list[param_idx]
        param_idx += 1

    for name, child in module._modules.items():
        if child is not None:
            param_idx = set_model_params(child, params_list, param_idx)

    return param_idx

## Step 1: Define A Model 
### - A perceptron (for a linear regression)

We want a simple model that will allow us to test the deployment of a model via pygrid

In [138]:
class LinearRegression(th.nn.Module):    
    
    def __init__(self):
        super(LinearRegression, self).__init__()
        self.linear = th.nn.Linear(3, 1)    
    
    def forward(self, x):
        y_pred = self.linear(x)
        return y_pred
    
model = LinearRegression()

In [139]:
[param.nelement() for param in model.parameters()]

[3, 1]

## Step 2: Define the training Plan

### Loss Function

In [140]:
def mse_with_logits(logits, targets, batch_size):
    """ Calculates softmax entropy
        Args:
            * logits: (NxC) outputs of dense layer
            * targets: (NxC) one-hot encoded labels
            * batch_size: value of N, temporarily required because Plan cannot trace .shape
    """
    return (logits - targets).sum() / batch_size

### Optimizer

In [141]:
def naive_sgd(param, **kwargs):
    return param - kwargs['lr'] * param.grad

### Training Procedure

In [142]:
@sy.func2plan()
def training_plan(X, y, batch_size, lr, model_params):
    # inject params into model
    set_model_params(model, model_params)

    # forward pass
    logits = model.forward(X)
    
    # loss
    loss = mse_with_logits(logits, y, batch_size)
    
    # backprop
    loss.backward()

    # step
    updated_params = [
        naive_sgd(param, lr=lr)
        for param in model_params
    ]
    
    # accuracy
    pred = th.argmax(logits, dim=1)
    target = th.argmax(y, dim=1)
    acc = pred.eq(target).sum().float() / batch_size

    return (
        loss,
        acc,
        *updated_params
    )

In [143]:
# Dummy input parameters to make the trace
model_params = [param.data for param in model.parameters()]  # raw tensors instead of nn.Parameter

X = th.randn(1, 3)
y = nn.functional.one_hot(th.tensor([2]))

lr = th.tensor([0.2])
batch_size = th.tensor([3])

_ = training_plan.build(X, y, batch_size, lr, model_params, trace_autograd=True)

In [144]:
model_params = [param.data for param in model.parameters()]  # raw tensors instead of nn.Parameter
X = th.randn(1, 1)
y = nn.functional.one_hot(th.tensor([2]))
lr = th.tensor([0.2])
batch_size = th.tensor([3])

_ = training_plan.build(X, y, batch_size, lr, model_params, trace_autograd=True)

RuntimeError: size mismatch, m1: [1 x 1], m2: [3 x 1] at ../aten/src/TH/generic/THTensorMath.cpp:136

In [ ]:
@sy.func2plan()
def avg_plan(avg, item, num):
    new_avg = []
    for i, param in enumerate(avg):
        new_avg.append((avg[i] * num + item[i]) / (num + 1))
    return new_avg

# Build the Plan
_ = avg_plan.build(model_params, model_params, th.tensor([1.0]))

In [ ]:
gridAddress = "pygri-pygri-frtwp3inl2zq-2ea21a767266378c.elb.us-east-1.amazonaws.com:5000"

In [ ]:
# PyGrid Node address

grid = ModelCentricFLClient(id="test", address=gridAddress, secure=False)
grid.connect() # These name/version you use in worker

In [ ]:
name = "perceptron" 
version = "1.0"

client_config = {
    "name": name,
    "version": version,
    "batch_size": 1,
    "lr": 0.2,
    "max_updates": 100  # custom syft.js option that limits number of training loops per worker
}

server_config = {
    "min_workers": 1,

    "max_workers": 5,
    "pool_selection": "random",
    "do_not_reuse_workers_until_cycle": 6,
    "cycle_length": 28800,  # max cycle length in seconds
    "num_cycles": 5,  # max number of cycles
    "max_diffs": 1,  # number of diffs to collect before avg
    "minimum_upload_speed": 0,
    "minimum_download_speed": 0,
    "iterative_plan": True  # tells PyGrid that avg plan is executed per diff
}

In [ ]:
model_params_state = State(
    state_placeholders=[
        PlaceHolder().instantiate(param)
        for param in model_params
    ]
)

response = grid.host_federated_training(
    model=model_params_state,
    client_plans={'training_plan': training_plan},
    client_protocols={},
    server_averaging_plan=avg_plan,
    client_config=client_config,
    server_config=server_config
)

In [ ]:
print("Host response:", response)